# Uniendo DataFrames (la práctica)

In [60]:
import pandas as pd
import numpy as np

Primero, vamos a declarar un par de DataFrames.

In [61]:
df_1 = pd.DataFrame(
    {
        key: [key + str(value) for value in range(0, 3)] for key in ['A', 'B', 'C', 'D'] 
    }
)
df_1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2


In [62]:
df_2 = pd.DataFrame(
    {
        key: [key + str(value) for value in range(4, 8)] for key in ['A', 'B', 'C', 'D'] 
    }
)
df_2

,A,B,C,D
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


## Concatenar DataFrames

Ahora, vamos a concatenar los DataFrames, usando ``pd.concat()``. Por defecto, ``pd.concat()`` hace la concatenación por el eje 0, es decir, a nivel de filas.

In [63]:
pd.concat([df_1, df_2])

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
0,A4,B4,C4,D4
1,A5,B5,C5,D5
2,A6,B6,C6,D6
3,A7,B7,C7,D7


Aquí se están repitiendo los índices, lo que es indeseable. Para indicarle a Pandas que no haga esto, podemos pasarle a la función ``pd.concat()`` el parámetro ``ignore_index=True``.

In [64]:
pd.concat([df_1, df_2], ignore_index=True)

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A4,B4,C4,D4
4,A5,B5,C5,D5
5,A6,B6,C6,D6
6,A7,B7,C7,D7


También, se le puede indicar que haga la concatenación a nivel de columnas.

In [65]:
pd.concat([df_1, df_2], axis=1)

,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,A4,B4,C4,D4
1,A1,B1,C1,D1,A5,B5,C5,D5
2,A2,B2,C2,D2,A6,B6,C6,D6
3,NaN,NaN,NaN,NaN,A7,B7,C7,D7


Como los DataFrames no tienen la misma cantidad de registros, se producen nulos. Estos los podemos remover con la función ``dropna()``.

In [66]:
pd.concat([df_1, df_2], axis=1).dropna()

,A,B,C,D,A,B,C,D
0,A0,B0,C0,D0,A4,B4,C4,D4
1,A1,B1,C1,D1,A5,B5,C5,D5
2,A2,B2,C2,D2,A6,B6,C6,D6


## Unir (mergear) DataFrames

In [67]:
left = pd.DataFrame(
    {
        key: [key + str(value) for value in range(0, 3)] for key in ['key', 'A', 'B', 'C'] 
    }
)
left

,key,A,B,C
0,key0,A0,B0,C0
1,key1,A1,B1,C1
2,key2,A2,B2,C2


In [68]:
right = pd.DataFrame(
    {
        key: [key + str(value) for value in range(0, 3)] for key in ['key', 'D', 'F', 'G']       
    }
)
right

,key,D,F,G
0,key0,D0,F0,G0
1,key1,D1,F1,G1
2,key2,D2,F2,G2


Siempre que vayamos a ejecutar un merge, el de la izquierda va a ser el DataFrame desde el cual se ejecuta el frame. Esto lo vemos aquí:

In [69]:
left.merge(right)

,key,A,B,C,D,F,G
0,key0,A0,B0,C0,D0,F0,G0
1,key1,A1,B1,C1,D1,F1,G1
2,key2,A2,B2,C2,D2,F2,G2


Por defecto, Pandas busca una columna en común, y hace el merge por esa columna. Sin embargo, es una buena práctica indicar la columna por la cuál se va a hacer el merge pasando el parámetro ``on``.

In [70]:
left.merge(right, on='key')

,key,A,B,C,D,F,G
0,key0,A0,B0,C0,D0,F0,G0
1,key1,A1,B1,C1,D1,F1,G1
2,key2,A2,B2,C2,D2,F2,G2


En caso de que el nombre de las columnas no coincida, se le puede indicar cuál columna tomar por cada DataFrame. Para esto, vamos a renombrar una columna de ``right``

In [77]:
right.rename(columns={'key': 'key2'}, inplace=True)
right

,key2,D,F,G
0,key0,D0,F0,G0
1,key1,D1,F1,G1
2,key2,D2,F2,G2


In [78]:
try:
    left.merge(right, on='key')
except KeyError as err:
    print('Error:', err)

Error: 'key'


In [79]:
left.merge(right, left_on='key', right_on='key2')

,key,A,B,C,key2,D,F,G
0,key0,A0,B0,C0,key0,D0,F0,G0
1,key1,A1,B1,C1,key1,D1,F1,G1
2,key2,A2,B2,C2,key2,D2,F2,G2


Ahora, ¿qué pasa si hay un nulo en una llave e intentamos hacer el merge?

In [84]:
right.iloc[0, 0] = np.nan
right

,key2,D,F,G
0,NaN,NaN,F0,G0
1,key1,D1,F1,G1
2,key2,D2,F2,G2


In [85]:
left.merge(right, left_on='key', right_on='key2')

,key,A,B,C,key2,D,F,G
0,key1,A1,B1,C1,key1,D1,F1,G1
1,key2,A2,B2,C2,key2,D2,F2,G2


Se descarta este dato. Esto se produce porque, por defecto, ``DataFrame.merge()`` une los conjuntos usando inner join.

In [86]:
left.merge(right, left_on='key', right_on='key2', how='left')

,key,A,B,C,key2,D,F,G
0,key0,A0,B0,C0,NaN,NaN,NaN,NaN
1,key1,A1,B1,C1,key1,D1,F1,G1
2,key2,A2,B2,C2,key2,D2,F2,G2
